# Oblivious Transfer

Oblivious Transfer (OT) is a fundamental cryptographic primitive in which a sender transfers one of several pieces of information to a receiver, but remains oblivious as to which piece has been transferred. This is a cornerstone of secure two-party computation.

:::{note}
**Reference**: Based on Sections 8-9 of "How To Simulate It" by Yehuda Lindell (2016).
:::

## What is Oblivious Transfer?
:label: ot-intro

### The 1-out-of-2 Oblivious Transfer

In **1-out-of-2 OT**, denoted $OT_2^1$:

- **Sender's input**: Two strings $(s_0, s_1)$
- **Receiver's input**: A choice bit $\sigma \in \{0, 1\}$
- **Receiver's output**: $s_\sigma$ (the string corresponding to its choice)
- **Sender's output**: Nothing (remains oblivious to $\sigma$)

### Security Requirements

1. **Receiver privacy**: The sender learns nothing about $\sigma$
2. **Sender privacy**: The receiver learns nothing about $s_{1-\sigma}$ (the string not chosen)

### The Ideal Functionality

The ideal $\mathcal{F}_{OT}$ functionality:
1. Receives $(s_0, s_1)$ from the sender
2. Receives $\sigma$ from the receiver
3. Sends $s_\sigma$ to the receiver
4. Sends nothing to the sender

## Why Oblivious Transfer is Important

:::{important} Fundamental Result
Oblivious Transfer is **complete for secure two-party computation**: any two-party functionality can be securely computed given access to OT (along with commitments and zero-knowledge proofs).
:::

This makes OT one of the most important primitives in cryptography.

## Protocol 8.2: Oblivious Transfer (Semi-Honest)
:label: ot-protocol

We present a protocol based on the DDH assumption.

:::{dropdown} **Protocol: 1-out-of-2 OT**

**Public parameters**: Cyclic group $\mathbb{G}$ of prime order $q$ with generator $g$

**Sender input**: $(s_0, s_1) \in \{0,1\}^n \times \{0,1\}^n$

**Receiver input**: $\sigma \in \{0,1\}$

**The Protocol**:

1. **Receiver prepares public keys**:
   - Choose random $r \leftarrow \mathbb{Z}_q$
   - If $\sigma = 0$: Set $pk_0 = g^r$ and $pk_1 = h$ (random element)
   - If $\sigma = 1$: Set $pk_0 = h$ (random element) and $pk_1 = g^r$
   - Send $(pk_0, pk_1)$ to sender

2. **Sender encrypts both strings**:
   - Choose random $s \leftarrow \mathbb{Z}_q$
   - Compute $c_0 = (g^s, pk_0^s \cdot H(g^s) \oplus s_0)$
   - Compute $c_1 = (g^s, pk_1^s \cdot H(g^s) \oplus s_1)$
   - Send $(c_0, c_1)$ to receiver

3. **Receiver decrypts**:
   - Parse $c_\sigma = (\alpha, \beta)$
   - Compute $s_\sigma = \beta \oplus H(\alpha^r)$
   - Output $s_\sigma$

**Correctness**:
- If $\sigma = 0$: $\alpha^r = (g^s)^r = (pk_0)^s$, so decryption recovers $s_0$
- If $\sigma = 1$: $\alpha^r = (g^s)^r = (pk_1)^s$, so decryption recovers $s_1$
:::

### Security Intuition

- **Receiver privacy**: Under DDH, $(g, g^r, h, h^r)$ is indistinguishable from $(g, g^r, h, R)$ where $R$ is random. So the sender cannot tell which of $pk_0, pk_1$ corresponds to $g^r$.

- **Sender privacy**: The receiver can only decrypt $c_\sigma$ (because it only knows the discrete log of $pk_\sigma$). The other ciphertext $c_{1-\sigma}$ is semantically secure.

## Simulation for Corrupted Sender
:label: sim-sender

When the sender is corrupted, the simulator must:
1. **Extract** the sender's inputs $(s_0, s_1)$
2. Generate a view for the sender that hides the receiver's choice $\sigma$

:::{dropdown} **Simulator $\mathcal{S}_S$ (Corrupted Sender)**

**Goal**: Simulate the receiver's message without revealing $\sigma$

**Simulation**:
1. Choose random $r_0, r_1 \leftarrow \mathbb{Z}_q$
2. Set $pk_0 = g^{r_0}$ and $pk_1 = g^{r_1}$ (both are valid public keys)
3. Send $(pk_0, pk_1)$ to the adversarial sender $\mathcal{A}_S$
4. Receive $(c_0, c_1)$ from $\mathcal{A}_S$
5. **Extract**: Decrypt both ciphertexts using $r_0$ and $r_1$ to obtain $(s_0, s_1)$
6. Send $(s_0, s_1)$ to the ideal functionality $\mathcal{F}_{OT}$

**Analysis**:
- In the real protocol, one of $pk_0, pk_1$ is $g^r$ and the other is random
- In the simulation, both are of the form $g^r$ (but the sender doesn't know which is which)
- Under DDH, these distributions are computationally indistinguishable
- The simulator successfully extracts both inputs and produces an indistinguishable view
:::

## Simulation for Corrupted Receiver
:label: sim-receiver

When the receiver is corrupted, the simulator must:
1. **Extract** the receiver's choice bit $\sigma$
2. Generate encryptions without knowing $s_{1-\sigma}$ (the unchosen string)

:::{dropdown} **Simulator $\mathcal{S}_R$ (Corrupted Receiver)**

**Challenge**: Must encrypt $s_{1-\sigma}$ without knowing it

**Simulation**:
1. Receive $(pk_0, pk_1)$ from adversarial receiver $\mathcal{A}_R$
2. **Extract $\sigma$**: Check which of $pk_0, pk_1$ the adversary knows the discrete log for
   - This requires checking if $\mathcal{A}_R$ can decrypt trial encryptions
   - Or use the discrete log relation to determine which is $g^r$
3. Send $\sigma$ to ideal functionality and receive $s_\sigma$
4. Encrypt $s_\sigma$ under $pk_\sigma$ as in the real protocol
5. Encrypt a **random string** under $pk_{1-\sigma}$ (since receiver cannot decrypt it)
6. Send $(c_0, c_1)$ to $\mathcal{A}_R$

**Analysis**:
- The encryption under $pk_{1-\sigma}$ is semantically secure (receiver doesn't have the secret key)
- Therefore, encrypting a random string vs. encrypting $s_{1-\sigma}$ are indistinguishable
- The simulator produces an indistinguishable view using only $\sigma$ and $s_\sigma$
:::

## The Extraction Challenge
:label: extraction

A key challenge in simulating the corrupted receiver is **extracting the choice bit $\sigma$**.

### Why Extraction is Needed

The simulator must:
- Determine what input the adversarial receiver is using
- Send this input to the ideal functionality
- Receive back the corresponding output

Without extraction, the simulator cannot determine which value the adversary should receive.

### Extraction Techniques

Several approaches for extraction:

1. **Discrete log extraction**: If the receiver's message reveals information about which $pk$ it knows the discrete log of

2. **Trapdoor extraction**: Use a trapdoor in the public parameters to extract the adversary's choice

3. **Zero-knowledge proofs**: Require the receiver to prove knowledge of the discrete log of exactly one of $pk_0, pk_1$

## Common Reference String (CRS) Model
:label: crs-model

Many protocols achieve security by using a **Common Reference String**.

### What is the CRS Model?

In the CRS model:
- Before the protocol starts, a trusted party generates public parameters (the CRS)
- All parties have access to this CRS
- The CRS is generated once and used for multiple protocol executions

### CRS vs. Plain Model

| Aspect | Plain Model | CRS Model |
|--------|-------------|------------|
| Setup | None | Trusted setup generates CRS |
| Security | Harder to achieve | Enables stronger protocols |
| Examples | Coin tossing (semi-honest) | UC-secure protocols, NIZKs |
| Simulation | Often requires rewinding | Trapdoor in CRS aids simulation |

### Protocol 9.1: OT in the CRS Model

:::{note} Enhanced OT Protocol with CRS

**Common Reference String**: $(g, h = g^\alpha)$ where $\alpha$ is unknown to all parties

**For simulation**: The simulator knows the trapdoor $\alpha$

**Modified Protocol**:
- Receiver sends $(pk_0, pk_1)$ along with a proof that it knows the discrete log of exactly one
- The CRS enables more efficient proofs and extraction

**Advantage**: The simulator can use $\alpha$ to extract the receiver's choice more efficiently
:::

### Why CRS Helps Simulation

1. **Trapdoor extraction**: The simulator can use the CRS trapdoor to extract adversary inputs without rewinding

2. **Non-interactive proofs**: Enables NIZKs (Non-Interactive Zero-Knowledge proofs)

3. **Straight-line simulation**: No need for rewinding, which is important for concurrent security

4. **Universal composability**: CRS model enables UC-secure protocols

## Security Against Malicious Adversaries
:label: malicious-security

The protocol described achieves semi-honest security. For malicious security:

### Challenges with Malicious Adversaries

A malicious sender could:
- Send invalid ciphertexts that leak information about the receiver's choice
- Not properly encrypt the values $(s_0, s_1)$

A malicious receiver could:
- Send invalid public keys that allow decrypting both values
- Know the discrete log of both $pk_0$ and $pk_1$

### Achieving Malicious Security

To handle malicious adversaries:

1. **Zero-knowledge proofs of correct behavior**:
   - Receiver proves it knows the discrete log of exactly one of $pk_0, pk_1$
   - Sender proves that ciphertexts are properly formed

2. **Cut-and-choose techniques**: Repeat the protocol multiple times and verify a random subset

3. **UC framework**: Use the universal composability framework with CRS

## Theorem 8.3: Security of OT Protocol
:label: ot-theorem

:::{important} Theorem 8.3
Assuming the DDH assumption holds, Protocol 8.2 securely computes the $OT_2^1$ functionality in the presence of semi-honest adversaries.
:::

:::{dropdown} **Proof Sketch**

**Corrupted Sender**:
- Simulator generates both $pk_0, pk_1$ as $g^{r_0}, g^{r_1}$ (instead of one random)
- By DDH, sender cannot distinguish this from the real protocol
- Simulator extracts $(s_0, s_1)$ by decrypting both ciphertexts

**Corrupted Receiver**:
- Simulator determines $\sigma$ from $(pk_0, pk_1)$
- Encrypts $s_\sigma$ correctly under $pk_\sigma$
- Encrypts random string under $pk_{1-\sigma}$
- By semantic security, receiver cannot distinguish random from $s_{1-\sigma}$

Both simulations produce computationally indistinguishable views, proving security.
:::

## Applications of Oblivious Transfer

OT is used in:

1. **Secure Two-Party Computation**: OT is the building block for Yao's garbled circuits and GMW protocol

2. **Private Information Retrieval**: User retrieves a database entry without revealing which one

3. **Private Set Intersection**: Two parties compute the intersection of their sets without revealing other elements

4. **Secure Auctions**: Bidders submit bids without revealing losing bids

5. **Anonymous Credentials**: Obtain credentials without revealing which credential is being obtained

## Summary

In this section, we learned:

1. **Oblivious Transfer** allows a receiver to obtain one of two values without the sender learning which one
2. OT is **complete for secure computation**: any two-party functionality can be built from OT
3. **Simulation challenges**: Must extract adversary inputs (especially the receiver's choice bit $\sigma$) and generate views without knowing unselected values
4. **DDH-based protocol**: Can be proven secure under the DDH assumption for semi-honest adversaries
5. **CRS model**: Using a Common Reference String enables more efficient extraction and stronger security properties
6. **Malicious security**: Requires additional zero-knowledge proofs to ensure correct behavior

Oblivious Transfer showcases advanced simulation techniques, particularly the extraction of adversary inputs, which is crucial for secure computation protocols.
